# Project 12: Web Requests, Caching, DataFrames and Scraping

## Your Information

At the start of each assignment, you will need to provide us your name and the name of the partner you worked with for this assignment (if you had one). Double click on the cell below or click once and hit enter to edit it. Replace "First Last" with your first name and last name. Replace "None" with the first and last name of your partner if you had one for this assignment. We ask for this information so we don't accuse you of cheating when your code looks like your partner's.

Please keep these lines commented so they don't cause an error.

In [3]:
# MY NAME: Hyokyung Kim

# My PARTNER'S NAME: None

## Imports

Every project will begin with some import statements. It's crucial that you run the cell below, otherwise we will not be able to grade your code and provide feedback to you.

In [4]:
# it is considered a good coding practice to place all import statements at the top of the notebook
# please place all your import statements in this cell if you need to import any more modules for this lab

# we have imported these modules for you
import requests
import os
import json
import pandas as pd
from bs4 import BeautifulSoup
import student_grader
student_grader.initialize(os.getcwd(), "p12")

<h2 style="color:red">Warning (Note on Academic Misconduct):</h2>

**IMPORTANT**: **P12 and P13 are two parts of the same data analysis.** You **cannot** switch project partners between these two projects. That is if you partner up with someone for P12, you have to sustain that partnership until the end of P13. Now may be a good time to review [our course policies](https://cs220.cs.wisc.edu/f24/syllabus.html).

Under any circumstances, **no more than two students are allowed to work together on a project** as mentioned in the course policies. If your code is flagged by our code similarity detection tools, **both partners will be responsible** for sharing/copying the code, even if the code is shared/copied by one of the partners with/from other non-partner student(s). Note that each case of plagiarism will be reported to the Dean of Students with a zero grade on the project. **If you think that someone cannot be your project partner then don’t make that student your lab partner.**

## Lab Portion

### Learning Objectives

In this lab, you will practice how to:

* use HTTP requests to download content from the internet,
* cache data onto your computer,
* construct and modify DataFrames to analyze datasets,
* use `BeautifulSoup` to parse web pages and extract useful information.

### Section 1: File handling with the `os` module

#### Lab Question 1: Fetch `rankings.json` from an internet URL

Use the `requests` library to fetch the file at this URL:

`https://cs220.cs.wisc.edu/projects/data/rankings.json`.

Make sure to call the appropriate function to **raise** an HTTPError if status code is not `200`. Then create a variable called `file_text` that saves the text of the response.

**Hint:** You can review the relevant lecture code here: [Mike](https://canvas.wisc.edu/courses/427075/files/folder/Mikes_Lecture_Notes/lec29_web_1), [Louis](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/f24/Louis_Lecture_Notes/29_Web1).

Points possible: 4.0

In [5]:
import requests

url = "https://cs220.cs.wisc.edu/projects/data/rankings.json"
response = requests.get(url)

response.raise_for_status()

file_text = response.text


In [6]:
student_grader.check("lab-q1", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q1...
Great job! You passed all test cases for this question.


True

#### Lab Question 2: Save `rankings.json` as a file

Open a file in write mode called `rankings.json`, and write the contents of the variable `file_text` into it. Make sure to **close** the file afterwards (unless you used a `with` block to open the file).

**Hint:** You can review the relevant lecture code here: [Mike](https://canvas.wisc.edu/courses/427075/files/folder/Mikes_Lecture_Notes/lec26_files_directories), [Louis](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/f24/Louis_Lecture_Notes/26_Files_and_Directories).

Points possible: 4.0

In [7]:
with open("rankings.json", "w") as file:
    file.write(file_text)


In [8]:
student_grader.check("lab-q2", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q2...
Great job! You passed all test cases for this question.


True

Note that the cell above only checked if the file had been created, and **not** whether it contains the correct data. You must check that yourself. Check your `lab-p12` directory in Finder (Mac) / Explorer (Windows). It should now have a file called `rankings.json`. **Manually open** this file and confirm that it contains the contents of the page [rankings.json](https://cs220.cs.wisc.edu/projects/data/rankings.json).

#### Function 1: `download`

Now, you will implement a function `download` to download data from the internet and save it to a file. 

This function takes in two arguments `url` and `filename`. The contents at the address pointed to by the `url` field should be saved into the file whose path is specified by `filename`. Remember that you can reuse the code you wrote above.

The naive version of function `download` described above has one big disadvantage: it **downloads** the file even if it has already been created. Fetching data from webpages takes both time and resources, and **must** be avoided as much as possible. In particular, repeatedly downloading files that have been already downloaded is a **very bad** coding practices, and **must** be avoided.

Therefore, we need to ensure the `download` function implements *caching*. This means that **before** downloading the file from the internet, the function **must** check if the file already exists. If the file already exists, the function should return the message `"<filename> already exists!"` where `filename` is the argument. It should **not** make a request. Don't forget the space between `<filename>` and the following word.

**Hint:** You can use the `os.path.exists` function to check if the `filename` already exists.

**Hint:** If you're struggling to pass the test, we've shown you the code that we are using to test your `download` function in lab question 3.

Points possible: 4.0

In [9]:
import os
import requests

def download(url, filename):
    # Check if the file already exists
    if os.path.exists(filename):
        return f"{filename} already exists!"
    
    # Make the HTTP request
    response = requests.get(url)
    response.raise_for_status()  # Raise HTTPError for bad responses
    
    # Get the text content
    file_text = response.text
    
    # Open the file with 'utf-8' encoding and write the text content
    with open(filename, "w", encoding="utf-8") as file:
        file.write(file_text)
    
    return f"{filename} created!"


In [10]:
student_grader.check("lab-function-download", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-function-download...
Great job! You passed all test cases for this question.


True

#### Lab Question 3: Test the `download` function

Below is the code we used to check if your `download` function is correct.

The cell checks whether the `download` function checks existing files, and whether it correctly downloads files from the given address. An `assert` statement checks if a boolean expression is true and throws an error if not.

**Do NOT modify the code in the cell below**. Think about why the test code is written in this way. Ask a TA if you're not sure.

Points possible: 4.0

In [11]:
# delete the file if it already exists and download the file
rankings_url = "https://cs220.cs.wisc.edu/projects/data/rankings.json"

if os.path.exists("rankings.json"):
    os.remove("rankings.json") # delete the existing file

return_result_1 = download(rankings_url, "rankings.json") # should be "rankings.json created!"
file_size_1 = os.path.getsize("rankings.json") # should be none-zero

f = open("rankings.json", "w") # rewrite the contents of the file
f.close()

return_result_2 = download(rankings_url, "rankings.json") # should be "rankings.json already exists!"
file_size_2 = os.path.getsize("rankings.json") # should be 0
os.remove("rankings.json")
return_result_3 = download(rankings_url, "rankings.json") # should be "rankings.json created!"

assert return_result_1 == "rankings.json created!"
assert return_result_2 == "rankings.json already exists!"
assert return_result_3 == "rankings.json created!"
assert file_size_1 > 0
assert file_size_2 == 0

In [12]:
student_grader.check("lab-q3", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q3...
Great job! You passed all test cases for this question.


True

You **must** use this `download` function to download files during P12. This will ensure that you do not download the files each time you 'Restart & Run All'.

### Section 2: Creating DataFrames

For this project, we will be analyzing statistics about world university rankings adapted from
[here](https://cwur.org/). The `rankings.json` file was created by scraping content from pages on the linked website. 

We are going to use `pandas` throughout the lab and project to analyze this dataset.

In lecture, you reviewed different ways to create pandas DataFrames. In this section, you will create a DataFrame `rankings` **by reading the JSON data** saved in `rankings.json`.

The output of the next cell must look like this:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Year</th>
      <th>World Rank</th>
      <th>Institution</th>
      <th>Country</th>
      <th>National Rank</th>
      <th>Education Rank</th>
      <th>Employability Rank</th>
      <th>Faculty Rank</th>
      <th>Research Rank</th>
      <th>Score</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>2021</td>
      <td>1</td>
      <td>Harvard University</td>
      <td>USA</td>
      <td>1</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>100.0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2021</td>
      <td>2</td>
      <td>Massachusetts Institute of Technology</td>
      <td>USA</td>
      <td>2</td>
      <td>4.0</td>
      <td>12.0</td>
      <td>2.0</td>
      <td>8.0</td>
      <td>96.7</td>
    </tr>
    <tr>
      <th>2</th>
      <td>2021</td>
      <td>3</td>
      <td>Stanford University</td>
      <td>USA</td>
      <td>3</td>
      <td>10.0</td>
      <td>4.0</td>
      <td>3.0</td>
      <td>2.0</td>
      <td>95.1</td>
    </tr>
    <tr>
      <th>3</th>
      <td>2021</td>
      <td>4</td>
      <td>University of Cambridge</td>
      <td>United Kingdom</td>
      <td>1</td>
      <td>3.0</td>
      <td>25.0</td>
      <td>4.0</td>
      <td>10.0</td>
      <td>94.1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>2021</td>
      <td>5</td>
      <td>University of Oxford</td>
      <td>United Kingdom</td>
      <td>2</td>
      <td>7.0</td>
      <td>27.0</td>
      <td>9.0</td>
      <td>4.0</td>
      <td>93.3</td>
    </tr>
  </tbody>
</table>

In [13]:
# we have done this one for you

rankings = pd.read_json('rankings.json')
rankings.head()

,Year,World Rank,Institution,Country,National Rank,Education Rank,Employability Rank,Faculty Rank,Research Rank,Score
0,2021,1,Harvard University,USA,1,1.0,1.0,1.0,1.0,100.0
1,2021,2,Massachusetts Institute of Technology,USA,2,4.0,12.0,2.0,8.0,96.7
2,2021,3,Stanford University,USA,3,10.0,4.0,3.0,2.0,95.1
3,2021,4,University of Cambridge,United Kingdom,1,3.0,25.0,4.0,10.0,94.1
4,2021,5,University of Oxford,United Kingdom,2,7.0,27.0,9.0,4.0,93.3


#### Lab Question 4: Find the unique universities in the dataset

As the dataset contains rankings for three different years, the same university may have featured multiple times. Find the names of the unique universities that are represented in the dataset.

First, extract just the names of the institutions as a `pandas` **Series**. Then, make a **list** of unique names called `institutions_list`. **Series** can be easily typecast just like any other data type in Python.

Points possible: 4.0

In [14]:
import pandas as pd

rankings = pd.read_json("rankings.json")  

institutions = rankings["Institution"]

institutions_list = list(set(institutions))


In [15]:
student_grader.check("lab-q4", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q4...
Great job! You passed all test cases for this question.


True

### Section 3: Use `value_counts` to count instances in a dataframe

Now, let's find the country that is the 5th most represented in the dataframe, and the number of times it features. Recall that `value_counts` enables us to count number of occurrences of unique values in a pandas **Series**.

#### Lab Question 5: Obtain the counts for all countries

First, use the `value_counts` function on the `Country` column of `rankings`, and then typecast to a pandas **Series** called `country_counts`. This **Series** should contain each country in the dataset and the number of times it occurs.

**Hint:** You can review the relevant lecture code here: [Mike](https://canvas.wisc.edu/courses/427075/files/folder/Mikes_Lecture_Notes/lec27_pandas_1), [Louis](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/f24/Louis_Lecture_Notes/27_Pandas1).

Points possible: 4.0

In [16]:
import pandas as pd

rankings = pd.read_json("rankings.json") 

country_counts = rankings["Country"].value_counts()

country_counts = pd.Series(country_counts)

fifth_country = country_counts.index[4]
fifth_count = country_counts.iloc[4]

print(f"The 5th most represented country is {fifth_country} with {fifth_count} occurrences.")


The 5th most represented country is France with 232 occurrences.


In [17]:
student_grader.check("lab-q5", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q5...
The 5th most represented country is France with 232 occurrences.
Great job! You passed all test cases for this question.


True

#### Lab Question 6: Find the 5th most represented country

Use the `.index` attribute of the **Series** `country_counts` to fetch the name of the 5th most represented country. Note that `country_counts` is **sorted** in *decreasing* order of the number of times each country appears in `rankings`. You **must** use `loc` to fetch the count of this country. Make sure to use the **Series** `country_counts` defined in Question 5.

**Hint**: The pandas `Series.index` works differently from the `.index` method you are familiar with for **lists**. `Series.index` takes in the numerical **index** of the element you want to access, and returns the **label** of the element. Then, you can pass the **label** to `.loc` to access the corresponding value in the `Series`.

Points possible: 4.0

In [18]:
# Find the name of the 5th most represented country
fifth_country = country_counts.index[4]

# Use .loc to fetch the count of this country
fifth_count = country_counts.loc[fifth_country]

# Print the results
print(fifth_country, fifth_count)


France 232


In [19]:
student_grader.check("lab-q6", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q6...
France 232
Great job! You passed all test cases for this question.


True

### Section 4: `loc` vs `iloc`

In this lab and project, you **must** only use `iloc` to access from a **DataFrame**. Using `loc` and explicitly hardcoding the `loc` of the row you want to access will be considered **hardcording**. This is because `iloc` selects rows and columns at the given **integer position** while `loc` selects rows at the given **pandas index**. 

Recall that **row index** can be given meaningful names like string indices. Consider a scenario where you add rows to the beginning of the DataFrame - if you use `.loc` indexing, your answer will become **incorrect** if the data changes. Whereas if you use `.iloc`, you will always get the correct answer.

This distinction may not be as intuitive for the current `rankings` **DataFrame**. As an example, use both `loc` and `iloc` to fetch the first row in `rankings`.

In [20]:
# use 'iloc' to extract the first row from 'rankings'

first_row_iloc = rankings.iloc[0]
first_row_iloc

Year                                2021
World Rank                             1
Institution           Harvard University
Country                              USA
National Rank                          1
Education Rank                       1.0
Employability Rank                   1.0
Faculty Rank                         1.0
Research Rank                        1.0
Score                              100.0
Name: 0, dtype: object

In [21]:
# use 'loc' to extract the first row from 'rankings'

first_row_loc = rankings.loc[0]
first_row_loc

Year                                2021
World Rank                             1
Institution           Harvard University
Country                              USA
National Rank                          1
Education Rank                       1.0
Employability Rank                   1.0
Faculty Rank                         1.0
Research Rank                        1.0
Score                              100.0
Name: 0, dtype: object

The results are exactly the same! This happens since the integer positions correspond to the pandas indices in the `rankings` dataframe. **However, this will not always hold true** - as we see in the next question.

#### Lab Question 7: Use boolean indexing to filter data

Now, use **boolean indexing** to extract data from the **DataFrame**. You can find the relevant lecture code here: [Mike](https://canvas.wisc.edu/courses/427075/files/folder/Mikes_Lecture_Notes/lec28_pandas_2), [Louis](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/f24/Louis_Lecture_Notes/28_Pandas2).

Create a **DataFrame** `rankings_arg_bra` that **only** consists of rankings of universities from *Argentina* or *Brazil*. 

**Hint**: When implementing **boolean indexing** in `pandas`, the `or` operator is represented by `|` and the `and` operator is represented by `&`. Remember that you should separate conditions with parentheses when performing boolean indexing with multiple conditions.

Points possible: 4.0

In [22]:
# Use boolean indexing to filter rows where the 'Country' column is 'Argentina' or 'Brazil'
rankings_arg_bra = rankings[(rankings["Country"] == "Argentina") | (rankings["Country"] == "Brazil")]

# Display the head of the resulting DataFrame
print(rankings_arg_bra.head())


     Year  World Rank                           Institution    Country  \
104  2021         105               University of São Paulo     Brazil   
346  2021         347                University of Campinas     Brazil   
355  2021         356            University of Buenos Aires  Argentina   
359  2021         360  Federal University of Rio de Janeiro     Brazil   
420  2021         421            São Paulo State University     Brazil   

     National Rank  Education Rank  Employability Rank  Faculty Rank  \
104              1           518.0               333.0         131.0   
346              2             NaN                 NaN           NaN   
355              1           319.0              1347.0           NaN   
359              3           445.0               440.0           NaN   
420              4             NaN                 NaN           NaN   

     Research Rank  Score  
104           85.0   81.5  
346          310.0   76.0  
355          324.0   75.9  
359       

In [23]:
student_grader.check("lab-q7", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q7...
     Year  World Rank                           Institution    Country  \
104  2021         105               University of São Paulo     Brazil   
346  2021         347                University of Campinas     Brazil   
355  2021         356            University of Buenos Aires  Argentina   
359  2021         360  Federal University of Rio de Janeiro     Brazil   
420  2021         421            São Paulo State University     Brazil   

     National Rank  Education Rank  Employability Rank  Faculty Rank  \
104              1           518.0               333.0         131.0   
346              2             NaN                 NaN           NaN   
355              1           319.0              1347.0           NaN   
359              3           445.0               440.0           NaN   
420              4             NaN                 NaN           NaN   

     Research Rank  Score  
104    

True

Now, we will try to extract the **first** value in this new **DataFrame** using `iloc` and `loc`. As you'll see, using `loc` will not work the same way it did before. In fact, it will throw an **error**. To verify, run the two cells below:

In [24]:
first_row_iloc = rankings_arg_bra.iloc[0]

first_row_iloc

Year                                     2021
World Rank                                105
Institution           University of São Paulo
Country                                Brazil
National Rank                               1
Education Rank                          518.0
Employability Rank                      333.0
Faculty Rank                            131.0
Research Rank                            85.0
Score                                    81.5
Name: 104, dtype: object

In [25]:
# uncomment this to see how .loc causes a KeyError
# first_row_loc = rankings_arg_bra.loc[0]

# first_row_loc

We see that using `.loc` now causes a **KeyError**.

`.loc[0]` tries to find the row with the *labeled* **index** 0. Actually, `rankings_arg_bra` starts at the labeled **index** 104. There is no 0. Hence the **KeyError**.

#### Lab Question 8: Sort the DataFrame

The **DataFrame** in Question 7 is sorted by `World Rank`, with the result that universities from *Argentina* and *Brazil* are interleaved throughout the data. **Re-sort** the data to sort by `Country` so that all universities from *Argentina* appear **first** followed by universities from *Brazil*. Within each country, the universities should be **sorted** by their `National Rank`. 

Use the `sort_values` function of `pandas`. You can find the relevant lecture code here: [Mike](https://canvas.wisc.edu/courses/427075/files/folder/Mikes_Lecture_Notes/lec27_pandas_1), [Louis](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/f24/Louis_Lecture_Notes/31_Web3). Remember - by default, `pandas` returns a **new** sorted **DataFrame** and does **not** modify the existing one.

Recall that `sort_values` takes an argument for the parameter `by` as the column name, based on which you want to do the sorting. If you want to use one column for primary sorting and another for secondary sorting, you can specify a **list** of column names.

Points possible: 4.0

In [26]:
import pandas as pd

# Load the dataset
rankings = pd.read_json("rankings.json")  # Ensure the JSON file is loaded into a DataFrame

# Filter the DataFrame for universities from Argentina or Brazil
rankings_arg_bra = rankings[(rankings["Country"] == "Argentina") | (rankings["Country"] == "Brazil")]

# Sort the filtered DataFrame by 'Country' and then by 'National Rank'
sorted_rankings_arg_bra = rankings_arg_bra.sort_values(by=["Country", "National Rank"])

# Display the sorted DataFrame
print(sorted_rankings_arg_bra)


      Year  World Rank                      Institution    Country  \
355   2021         356       University of Buenos Aires  Argentina   
2364  2022         365       University of Buenos Aires  Argentina   
4381  2023         382       University of Buenos Aires  Argentina   
620   2021         621  National University of La Plata  Argentina   
2638  2022         639  National University of La Plata  Argentina   
...    ...         ...                              ...        ...   
5938  2023        1939   Federal University of Amazonas     Brazil   
1944  2021        1945   Federal University of Amazonas     Brazil   
3962  2022        1963   Federal University of Amazonas     Brazil   
1995  2021        1996  Santa Catarina State University     Brazil   
3996  2022        1997   Federal University of Maranhão     Brazil   

      National Rank  Education Rank  Employability Rank  Faculty Rank  \
355               1           319.0              1347.0           NaN   
2364         

In [27]:
student_grader.check("lab-q8", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q8...
      Year  World Rank                      Institution    Country  \
355   2021         356       University of Buenos Aires  Argentina   
2364  2022         365       University of Buenos Aires  Argentina   
4381  2023         382       University of Buenos Aires  Argentina   
620   2021         621  National University of La Plata  Argentina   
2638  2022         639  National University of La Plata  Argentina   
...    ...         ...                              ...        ...   
5938  2023        1939   Federal University of Amazonas     Brazil   
1944  2021        1945   Federal University of Amazonas     Brazil   
3962  2022        1963   Federal University of Amazonas     Brazil   
1995  2021        1996  Santa Catarina State University     Brazil   
3996  2022        1997   Federal University of Maranhão     Brazil   

      National Rank  Education Rank  Employability Rank  Faculty Rank  \

True

### Section 5: Create a new, simplified DataFrame to track changes in rankings

As we have seen, universities that have featured in rankings of multiple years are featured repeatedly. To simplify comparisons, we want to feature each university once and remove all other metrics. 

This time - instead of simply ranking universities, we want to find the **absolute change** in universities' rankings between the years *2021* and *2022*. We are only interested in the absolute change and **not** whether the rank improved or declined.  

#### Lab Question 9: Find the absolute difference in `World Rank` for *"University of Madras"* between the `Year` *2021* and *2022*.

First, let's attempt to measure the change for one particular university, *University of Madras*.

Points possible: 4.0

In [28]:
# Filter the rows for "University of Madras" in the dataset
madras_2021 = rankings[(rankings["Institution"] == "University of Madras") & (rankings["Year"] == 2021)]
madras_2022 = rankings[(rankings["Institution"] == "University of Madras") & (rankings["Year"] == 2022)]

# Extract the rankings for 2021 and 2022 using .iloc[0]
rank_2021 = madras_2021["World Rank"].iloc[0]
rank_2022 = madras_2022["World Rank"].iloc[0]

# Calculate the absolute difference
absolute_diff_madras = abs(rank_2021 - rank_2022)

# Display the absolute difference
print(absolute_diff_madras)


59


In [29]:
student_grader.check("lab-q9", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q9...
59
Great job! You passed all test cases for this question.


True

#### Lab Question 10: Create a Series with the absolute difference in ranks for *"University of Madras"* between *2021* and *2022*

Create a **dictionary** with the keys as the **strings** `Institution` and `Absolute Change`. The values should be the relevant values for *University of Madras*. You should use the `absolute_diff_madras` variable from the previous question. Then, convert this **dictionary** to a **Series**.

Points possible: 4.0

In [30]:
# Define the dictionary with Institution and Absolute Change
madras_dict = {
    "Institution": "University of Madras",
    "Absolute Change": absolute_diff_madras
}

# Convert the dictionary to a pandas Series
madras_series = pd.Series(madras_dict)

# Display the Series
print(madras_series)


Institution        University of Madras
Absolute Change                      59
dtype: object


In [31]:
student_grader.check("lab-q10", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q10...
Institution        University of Madras
Absolute Change                      59
dtype: object
Great job! You passed all test cases for this question.


True

#### Lab Question 11: Create the change_in_rankings DataFrame

Create a **DataFrame** `change_in_rankings` with just 2 columns, `Institution` and `Absolute Change` where **each** university is only featured once. If the institution is **not** present in the rankings of **both** years, we will just ignore it.

The institutions should be **sorted** in *increasing* order of their **absolute change**. For institutions with the **same** absolute change, sort them *alphabetically* by their **names**.

**Warning:** Even if your code is optimal, this cell may take a few seconds to run. However, if it takes much longer than that (say, if it takes 30 seconds or longer), then you will **need** to optimize your code so it runs faster.

Points possible: 4.0

In [34]:
# Initialize an empty list to store dictionaries for each institution
institutions_data = []

# Loop through the list of unique institutions
for institution in institutions_list:
    # Filter the DataFrame for the current institution
    institution_data = rankings[rankings["Institution"] == institution]
    
    # Get the list of years in which the institution appears
    years = institution_data["Year"].tolist()
    
    # Skip if the institution is not present in both 2021 and 2022
    if 2021 not in years or 2022 not in years:
        continue
    
    # Extract the World Rank for 2021 and 2022
    rank_2021 = institution_data[institution_data["Year"] == 2021]["World Rank"].iloc[0]
    rank_2022 = institution_data[institution_data["Year"] == 2022]["World Rank"].iloc[0]
    
    # Calculate the absolute change in ranking
    absolute_change = abs(rank_2021 - rank_2022)
    
    # Create a dictionary with Institution and Absolute Change
    institution_dict = {
        "Institution": institution,
        "Absolute Change": absolute_change
    }
    
    # Append the dictionary to the list
    institutions_data.append(institution_dict)

# Convert the list of dictionaries to a DataFrame
change_in_rankings = pd.DataFrame(institutions_data)

# Sort the DataFrame by Absolute Change (ascending) and Institution (alphabetical order for ties)
change_in_rankings = change_in_rankings.sort_values(by=["Absolute Change", "Institution"])

# Display the resulting DataFrame
change_in_rankings


,Institution,Absolute Change
1310,Ben-Gurion University of the Negev,0
886,Brown University,0
1628,Carleton University,0
1684,Central Queensland University,0
1289,Complutense University of Madrid,0
...,...,...
1878,Sechenov University,335
1410,Duy Tân University,342
777,Huzhou University,398
96,SRM Institute of Science and Technology,405


In [35]:
student_grader.check("lab-q11", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q11...
Great job! You passed all test cases for this question.


True

### Section 6: BeautifulSoup

As mentioned in Section 2, the `rankings.json` file was created by parsing HTML content on the Web, specifically the web pages listed below.

* https://cs220.cs.wisc.edu/projects/data/2021.html
* https://cs220.cs.wisc.edu/projects/data/2022.html
* https://cs220.cs.wisc.edu/projects/data/2023.html

Now, let's write a function to do this ourselves. We will use the `BeautifulSoup` module to scrape web pages and extract information. It is recommended that you review the relevant lecture code here: [Mike](https://canvas.wisc.edu/courses/427075/files/folder/Mikes_Lecture_Notes/lec31_web_3), [Louis](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/s24/Louis_Lecture_Notes/31_Web3).

#### Download `2021.html`, `2022.html` and `2023.html`

Use the `download` function you previously created to download the contents of each of the URLs above and save them into files. Name the files `2021.html`, `2022.html` and `2023.html` based on the respective URL.

Points possible: 4.0

In [36]:
# Use the 'download' function to download and save the HTML files
message_2021 = download("https://cs220.cs.wisc.edu/projects/data/2021.html", "2021.html")
message_2022 = download("https://cs220.cs.wisc.edu/projects/data/2022.html", "2022.html")
message_2023 = download("https://cs220.cs.wisc.edu/projects/data/2023.html", "2023.html")

# Print the messages returned by the 'download' function
print(message_2021)
print(message_2022)
print(message_2023)


2021.html created!
2022.html created!
2023.html created!


In [37]:
student_grader.check("lab-downloads", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-downloads...
2021.html already exists!
2022.html already exists!
2023.html already exists!
Great job! You passed all test cases for this question.


True

Note that the cell above only checked if the files had been created, and **not** whether they contain the correct data. You must check that yourself. Check your `lab-p12` directory in Finder (Mac) / Explorer (Windows). It should now have a files called `2021.html`, `2022.html` and `2023.html`. **Manually open** these files and confirm they contain the contents of the respective page:
* [2021.html](https://cs220.cs.wisc.edu/projects/data/2021.html)
* [2022.html](https://cs220.cs.wisc.edu/projects/data/2022.html)
* [2023.html](https://cs220.cs.wisc.edu/projects/data/2023.html)

#### Lab Question 12: Read `2021.html`

First, read the contents of the file `2021.html`.

**Hint:** If you get a `UnicodeDecodeError`, make sure all your calls to `open` have the keyword argument `encoding="utf-8"`.

Points possible: 4.0

In [38]:
# Open the file "2021.html", read its content, and store it in the variable 'file_content_2021'
with open("2021.html", "r", encoding="utf-8") as file:
    file_content_2021 = file.read()


In [39]:
student_grader.check("lab-q12", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q12...
Great job! You passed all test cases for this question.


True

#### Lab Question 13: Initialize `BeautifulSoup` object instance

Use the variable `file_content_2021` defined in Question 12 to create a `BeautifulSoup` object instance. You can review the relevant lecture code here: [Mike](https://canvas.wisc.edu/courses/397655/files/folder/Mikes_Lecture_Notes/lec31_web_3), [Louis](https://git.doit.wisc.edu/cdis/cs/courses/cs220/cs220-lecture-material/-/tree/main/s24/Louis_Lecture_Notes/31_Web3).

Note: the grader for this question only tests whether the variable you created is a `BeautifulSoup` object. It does not check if the variable has the correct value. We will check the correctness of the variable by finding certain tags in the following questions.

Points possible: 4.0

In [40]:
from bs4 import BeautifulSoup

# Create a BeautifulSoup object instance
soup_2021 = BeautifulSoup(file_content_2021, "html.parser")


In [41]:
student_grader.check("lab-q13", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q13...
Great job! You passed all test cases for this question.


True

#### Lab Question 14: Find the `table` element

The webpage has a `table` containing all the data we're trying to extract. Write the code to **find** this element. Use the variable `soup_2021` defined in Question 13.

Note: the grader for this question only tests whether you found a tag from the `BeautifulSoup` object defined in Question 13. It does not check if you found the right tag. We will check the correctness of the table by further finding all its header cells in Question 15.

Points possible: 5.0

In [42]:
from bs4 import BeautifulSoup

# Step 1: Open and read the content of "2021.html"
with open("2021.html", "r", encoding="utf-8") as file:
    file_content_2021 = file.read()

# Step 2: Create a BeautifulSoup object
soup_2021 = BeautifulSoup(file_content_2021, "html.parser")

# Step 3: Find the table element in the BeautifulSoup object
table_2021 = soup_2021.find("table")

# The variable 'table_2021' now contains the first <table> element from the HTML file.


In [43]:
student_grader.check("lab-q14", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q14...
Great job! You passed all test cases for this question.


True

#### Lab Question 15: Find all `th` tags, to parse the table header

Use the `table_2021` defined in Question 14 to find the table header. Remember that the table header is represented by the `th` tag. You can use the `find_all` function to find all instances of the given tag in a `BeautifulSoup` object.

**Hint**: The **header** should be a **list** of elements, that can be obtained by using the `get_text` method for each `th` element in the table. You may also find list comprehension useful here.

Points possible: 5.0

In [44]:
# Find all <th> tags in the table
th_tags = table_2021.find_all("th")

# Extract the text content of each <th> tag
header_2021 = [th.get_text(strip=True) for th in th_tags]

# Display the header
print(header_2021)


['World Rank', 'Institution', 'Country', 'National Rank', 'Education Rank', 'Employability Rank', 'Faculty Rank', 'Research Rank', 'Score']


In [45]:
student_grader.check("lab-q15", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q15...
['World Rank', 'Institution', 'Country', 'National Rank', 'Education Rank', 'Employability Rank', 'Faculty Rank', 'Research Rank', 'Score']
Great job! You passed all test cases for this question.


True

#### Lab Question 16: Build row dictionary for one row

Scrape the **second** row (the first one is the **header**!) of the `table`, convert the data to the appropriate **data types**, and populate the data into a **dictionary**. The **keys** of the dictionary **must** be the columns in the **DataFrame**. You **may** *hardcode* these **keys**, however it would be more efficient to use the variable `header_2021` obtained in the previous Question.

**Hint**: Rows can be found by locating the `tr` elements in the table. After identifying the second row of the table, try to figure out which tag separates the different columns.

The required data types for each column is:

|**Column Name**|**Data Type**|
|---------------|-------------|
|`World Rank`|**int**|
|`Institution`|**str**|
|`Country`|**str**|
|`National Rank`|**int**|
|`Education Rank`|**int**|
|`Employability Rank`|**int**|
|`Faculty Rank`|**int**|
|`Research Rank`|**int**|
|`Score`|**float**|

You can **compare** your output with the data in `rankings.json`, to confirm whether you have parsed the file correctly (note that you do **not** yet have to implement the `Year` column in your **dictionary**).

Points possible: 5.0

In [62]:
# find the second row of the table, parse it, and store the result in the variable 'row_dict'
rows = table_2021.find_all("tr")
second_row = rows[1]
cells = second_row.find_all("td")
row_dict = {}
column_types = {
    "World Rank": int, 
    "Institution": str,
    "Country": str,
    "National Rank": int,
    "Education Rank": int,
    "Employability Rank": int,
    "Faculty Rank": int,
    "Research Rank": int,
    "Score": float
}

for index, cell in enumerate(cells):
    column_name = header_2021[index]
    cell_value = cell.get_text().strip()
    if cell_value == "-":
        row_dict[column_name] = None
    else:
        row_dict[column_name] = column_types[column_name](cell_value)
        
print(row_dict)

{'World Rank': 1, 'Institution': 'Harvard University', 'Country': 'USA', 'National Rank': 1, 'Education Rank': 1, 'Employability Rank': 1, 'Faculty Rank': 1, 'Research Rank': 1, 'Score': 100.0}


In [63]:
student_grader.check("lab-q16", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q16...
{'World Rank': 1, 'Institution': 'Harvard University', 'Country': 'USA', 'National Rank': 1, 'Education Rank': 1, 'Employability Rank': 1, 'Faculty Rank': 1, 'Research Rank': 1, 'Score': 100.0}
Great job! You passed all test cases for this question.


True

#### Lab Question 17: Build list of all row dictionaries

Continuing from Question 16, scrape **all** rows in `table_2021`, **convert** data to appropriate types, and populate data into a row **dictionary** and append row all dictionaries to a **list**. You can use a loop to extract all rows and populate the list.

**Important**:
* Some fields in the dataset have **missing** values, represented simply as `"-"`. Such **missing** values should be replaced by `None` in your **dictionary**.
* Do NOT display the entire list as it can be very long and cause the notebook to run slow. You can print out the first few elements of the list to check if it matches the table inside the `2021.html` file.

Points possible: 5.0

In [64]:
# build a list of row dictionaries using the algorithm in Question 16, and store the result in 'rows_2021'
rows_2021 = []
all_rows = table_2021.find_all("tr")[1:]
column_types = {
    "World Rank": int, 
    "Institution": str,
    "Country": str,
    "National Rank": int,
    "Education Rank": int,
    "Employability Rank": int,
    "Faculty Rank": int,
    "Research Rank": int,
    "Score": float
}

for row in all_rows:
    row_dict = {}
    cells = row.find_all("td")
    if len(cells) != len(header_2021):
        continue
    for index, cell in enumerate(cells):
        column_name = header_2021[index]
        cell_value = cell.get_text().strip()
        if cell_value == "-":
            row_dict[column_name] = None
        else:
            row_dict[column_name] = column_types[column_name](cell_value)
    rows_2021.append(row_dict)
print(len(rows_2021))
print(rows_2021[:5])


2000
[{'World Rank': 1, 'Institution': 'Harvard University', 'Country': 'USA', 'National Rank': 1, 'Education Rank': 1, 'Employability Rank': 1, 'Faculty Rank': 1, 'Research Rank': 1, 'Score': 100.0}, {'World Rank': 2, 'Institution': 'Massachusetts Institute of Technology', 'Country': 'USA', 'National Rank': 2, 'Education Rank': 4, 'Employability Rank': 12, 'Faculty Rank': 2, 'Research Rank': 8, 'Score': 96.7}, {'World Rank': 3, 'Institution': 'Stanford University', 'Country': 'USA', 'National Rank': 3, 'Education Rank': 10, 'Employability Rank': 4, 'Faculty Rank': 3, 'Research Rank': 2, 'Score': 95.1}, {'World Rank': 4, 'Institution': 'University of Cambridge', 'Country': 'United Kingdom', 'National Rank': 1, 'Education Rank': 3, 'Employability Rank': 25, 'Faculty Rank': 4, 'Research Rank': 10, 'Score': 94.1}, {'World Rank': 5, 'Institution': 'University of Oxford', 'Country': 'United Kingdom', 'National Rank': 2, 'Education Rank': 7, 'Employability Rank': 27, 'Faculty Rank': 9, 'Rese

In [65]:
student_grader.check("lab-q17", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q17...
2000
[{'World Rank': 1, 'Institution': 'Harvard University', 'Country': 'USA', 'National Rank': 1, 'Education Rank': 1, 'Employability Rank': 1, 'Faculty Rank': 1, 'Research Rank': 1, 'Score': 100.0}, {'World Rank': 2, 'Institution': 'Massachusetts Institute of Technology', 'Country': 'USA', 'National Rank': 2, 'Education Rank': 4, 'Employability Rank': 12, 'Faculty Rank': 2, 'Research Rank': 8, 'Score': 96.7}, {'World Rank': 3, 'Institution': 'Stanford University', 'Country': 'USA', 'National Rank': 3, 'Education Rank': 10, 'Employability Rank': 4, 'Faculty Rank': 3, 'Research Rank': 2, 'Score': 95.1}, {'World Rank': 4, 'Institution': 'University of Cambridge', 'Country': 'United Kingdom', 'National Rank': 1, 'Education Rank': 3, 'Employability Rank': 25, 'Faculty Rank': 4, 'Research Rank': 10, 'Score': 94.1}, {'World Rank': 5, 'Institution': 'University of Oxford', 'Country': 'United Kingdom', 'Na

True

#### Function 2: `parse.html`

From question 12-17, we've succesfully located the table inside `2021.html` and parsed the table into a list of dictionaries. Naturally, we would want to generalize this process into a function so that it can work on the other html files, not just `2021.html`. This function `parse_html` should take in a `filename` as **input** and **return** a **list** of **dictionaries**, with each **dictionary** representing a **row** in the dataset.

Additionally, we **also** want to include the **key** `Year` to all our **dictionaries**. The `Year` value is **not** present in the dataset, so you need extract this value from the `filename`.

Finally, if you are having trouble passing the check for this function, move on to question 18, 19, and 20. Each question tests your `parse_html` function with increasingly difficult html files. Each test also comes with useful hints that can help you debug `parse_html`.

Points possible: 5.0

In [66]:
def parse_html(filename):
    '''parse_html(filename) parses an HTML file and 
    returns a list of dictionaries containing the tabular data'''
    
    with open(filename, "r", encoding="utf-8") as file:
        file_content = file.read()
    soup = BeautifulSoup(file_content, "html.parser")
    table = soup.find("table")
    if not table:
        return []  

    header_tags = table.find_all("th")
    header = [tag.get_text().strip() for tag in header_tags]

    year = int(filename.split("/")[-1].split(".")[0])

    column_types = {
        "World Rank": int, 
        "Institution": str,
        "Country": str,
        "National Rank": int,
        "Education Rank": int,
        "Employability Rank": int,
        "Faculty Rank": int,
        "Research Rank": int,
        "Score": float
    }
    
    rows_list = []
    all_rows = table.find_all("tr")[1:]  
    for row in all_rows:
        row_dict = {}
        cells = row.find_all("td")
        if len(cells) != len(header):
            continue  
        for index, cell in enumerate(cells):
            column_name = header[index]
            cell_value = cell.get_text().strip()

            if column_name == "World Rank":
                cell_value = cell_value.split()[0]  
                cell_value = "".join(c for c in cell_value if c.isdigit())  
            
            if column_name == "Institution":
                cell_value = cell_value.split("\n")[0].strip()

            if column_name in ["National Rank", "Education Rank", "Employability Rank", 
                               "Faculty Rank", "Research Rank"]:
                cell_value = "".join(c for c in cell_value if c.isdigit())
            elif column_name == "Score":
                cell_value = "".join(c for c in cell_value if c.isdigit() or c == ".")

            if cell_value == "":
                row_dict[column_name] = None
            else:
                try:
                    row_dict[column_name] = column_types[column_name](cell_value)
                except ValueError:
                    row_dict[column_name] = cell_value 

        row_dict["Year"] = year
        rows_list.append(row_dict)
    
    return rows_list

In [67]:
student_grader.check("lab-function-parse_html", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-function-parse_html...
Great job! You passed all test cases for this question.


True

#### Lab Question 18: Parse `2021.html`

We will now attempt to read `2021.html` using the `parse_html` function.

Points possible: 5.0

In [68]:
# Parse the 2021.html file using the parse_html function
parsed_2021 = parse_html("2021.html")

# Display the first 3 rows of the parsed data
print(parsed_2021[:3])


[{'World Rank': 1, 'Institution': 'Harvard University', 'Country': 'USA', 'National Rank': 1, 'Education Rank': 1, 'Employability Rank': 1, 'Faculty Rank': 1, 'Research Rank': 1, 'Score': 100.0, 'Year': 2021}, {'World Rank': 2, 'Institution': 'Massachusetts Institute of Technology', 'Country': 'USA', 'National Rank': 2, 'Education Rank': 4, 'Employability Rank': 12, 'Faculty Rank': 2, 'Research Rank': 8, 'Score': 96.7, 'Year': 2021}, {'World Rank': 3, 'Institution': 'Stanford University', 'Country': 'USA', 'National Rank': 3, 'Education Rank': 10, 'Employability Rank': 4, 'Faculty Rank': 3, 'Research Rank': 2, 'Score': 95.1, 'Year': 2021}]


In [69]:
student_grader.check("lab-q18", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q18...
[{'World Rank': 1, 'Institution': 'Harvard University', 'Country': 'USA', 'National Rank': 1, 'Education Rank': 1, 'Employability Rank': 1, 'Faculty Rank': 1, 'Research Rank': 1, 'Score': 100.0, 'Year': 2021}, {'World Rank': 2, 'Institution': 'Massachusetts Institute of Technology', 'Country': 'USA', 'National Rank': 2, 'Education Rank': 4, 'Employability Rank': 12, 'Faculty Rank': 2, 'Research Rank': 8, 'Score': 96.7, 'Year': 2021}, {'World Rank': 3, 'Institution': 'Stanford University', 'Country': 'USA', 'National Rank': 3, 'Education Rank': 10, 'Employability Rank': 4, 'Faculty Rank': 3, 'Research Rank': 2, 'Score': 95.1, 'Year': 2021}]
Great job! You passed all test cases for this question.


True

If you fail any of the checks above, go back and verify that you typecast properly. You could also manually open `2021.html`, and visually identify why your code failed.

#### Lab Question 19: Parse `2022.html`

We will now attempt to read `2022.html` using the `parse_html` function.

Points possible: 5.0

In [70]:
# run this cell to test whether 'parse_html' has been defined properly

parsed_2022 = parse_html("2022.html")

parsed_2022[:3]

[{'World Rank': 1,
  'Institution': 'Harvard University',
  'Country': 'USA',
  'National Rank': 1,
  'Education Rank': 1,
  'Employability Rank': 1,
  'Faculty Rank': 1,
  'Research Rank': 1,
  'Score': 100.0,
  'Year': 2022},
 {'World Rank': 2,
  'Institution': 'Massachusetts Institute of Technology',
  'Country': 'USA',
  'National Rank': 2,
  'Education Rank': 4,
  'Employability Rank': 12,
  'Faculty Rank': 2,
  'Research Rank': 7,
  'Score': 96.7,
  'Year': 2022},
 {'World Rank': 3,
  'Institution': 'Stanford University',
  'Country': 'USA',
  'National Rank': 3,
  'Education Rank': 11,
  'Employability Rank': 4,
  'Faculty Rank': 3,
  'Research Rank': 2,
  'Score': 95.1,
  'Year': 2022}]

In [71]:
student_grader.check("lab-q19", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q19...
Great job! You passed all test cases for this question.


True

It is **highly likely** that your code **failed the tests above**. To understand why,
1. Take a look at the value of `parsed_2022[0]['Institution']`.
2. **Manually open** `2022.html`, and **identify** what is going on there.

Then, go back to the definition of `parse_html` and **modify** it, so that it can deal with this case here. It is completely up to you, how you deal with this case here, as long as you correctly identify just the name of the `Institution` for all universities.

**Important**: While modifying `parse_html`, it is important that you do not break the function in such a way that it no longer works for `2021.html`. Your function **must** be able to read **both** `2021.html` and `2022.html`.

#### Lab Question 20:  Parse `2023.html`

We will now attempt to read `2023.html` using the `parse_html` function. As above, make sure to manually open the `2023.html` file and verify your output.

Points possible: 5.0

In [72]:
# run this cell to test whether 'parse_html' has been defined properly

parsed_2023 = parse_html("2023.html")

parsed_2023[:3]

[{'World Rank': 1,
  'Institution': 'Harvard University',
  'Country': 'USA',
  'National Rank': 1,
  'Education Rank': 1,
  'Employability Rank': 1,
  'Faculty Rank': 1,
  'Research Rank': 1,
  'Score': 100.0,
  'Year': 2023},
 {'World Rank': 2,
  'Institution': 'Massachusetts Institute of Technology',
  'Country': 'USA',
  'National Rank': 2,
  'Education Rank': 4,
  'Employability Rank': 12,
  'Faculty Rank': 3,
  'Research Rank': 9,
  'Score': 96.7,
  'Year': 2023},
 {'World Rank': 3,
  'Institution': 'Stanford University',
  'Country': 'USA',
  'National Rank': 3,
  'Education Rank': 11,
  'Employability Rank': 4,
  'Faculty Rank': 2,
  'Research Rank': 2,
  'Score': 95.2,
  'Year': 2023}]

In [73]:
student_grader.check("lab-q20", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for lab-q20...
Great job! You passed all test cases for this question.


True

### Lab Submission

Great work! You are now ready to submit lab-p12.
 **Submit your p12.ipynb on Gradescope to the lab-p12 assignment**, like usual. Remember that the grades for the lab portion of the project and the actual assignment grade are independent. You will submit the same notebook (at different levels of completion) to two different assignments.

## Project Portion

### Learning Objectives:

In this project, you will demonstrate your ability to

* read and write files,
* create and use `Pandas DataFrames`,
* use `BeautifulSoup` to parse web pages.

Please go through the lab portion before working on this project. The lab introduces some useful techniques related to this project.

<h3 style="color:red">Warning (Note on Academic Misconduct):</h3>


**IMPORTANT**: **P12 and P13 are two parts of the same data analysis.** You **cannot** switch project partners between these two projects. That is if you partner up with someone for P12, you have to sustain that partnership until the end of P13. Now may be a good time to review [our course policies](https://cs220.cs.wisc.edu/f24/syllabus.html).

Under any circumstances, **no more than two students are allowed to work together on a project** as mentioned in the course policies. If your code is flagged by our code similarity detection tools, **both partners will be responsible** for sharing/copying the code, even if the code is shared/copied by one of the partners with/from other non-partner student(s). Note that each case of plagiarism will be reported to the Dean of Students with a zero grade on the project. **If you think that someone cannot be your project partner then don’t make that student your lab partner.**

### Introduction:

For this project, you're going to analyze World University Rankings!

Specifically, you're going to use Pandas to analyze various statistics of the top ranked universities across the world, over the last three years.

**Important Warning:** Do **not** download any of the other `json` or `html` files manually (you **must** write Python code to download these automatically, as in Lab-P12). When we run the autograder, the other files such as `rankings.json`, `2021.html`, `2022.html`, `2023.html` will **not** be in the directory. So, unless your `p12.ipynb` downloads these files, the Gradescope autograder will **deduct** points from your public score. More details can be found in the **Setup** section of the project.

### Data:

For this project, we will be analyzing statistics about world university rankings adapted from [here](https://cwur.org/). These are the specific webpages that we extracted the data from:

* https://cwur.org/2020-21.php
* https://cwur.org/2021-22.php
* https://cwur.org/2023.php

Later in the project, you will be scraping these webpages and extracting the data yourself. Since we don't want all of you bombarding these webpages with requests, we have made snapshots of these webpages, and hosted them on GitLab. You can find the snapshots here:

* https://cs220.cs.wisc.edu/projects/data/2021.html
* https://cs220.cs.wisc.edu/projects/data/2022.html
* https://cs220.cs.wisc.edu/projects/data/2023.html

We have also tweaked the snapshots a little, to streamline the process of data extraction for you. You will be extracting the data from these three html pages and analyzing them. However, to make the start of the project a little easier, we have already parsed the files for you! We have gathered the data from these html files, and collected them in a single json file, which can be found here:

* https://cs220.cs.wisc.edu/projects/data/rankings.json

You will work with this json file for most of this project. At the end of this project, you will generate an identical json file by parsing the html files yourself.

### Project Requirements:

You **may not** hardcode indices in your code. You **may not** manually download **any** files for this project, unless you are **explicitly** told to do so. For all other files, you **must** use the `download` function to download the files.

**Store** your final answer for each question in the **variable specified for each question**. This step is important because Otter grades your work by comparing the value of this variable against the correct answer.

For some of the questions, we'll ask you to write (then use) a function to compute the answer. If you compute the answer **without** creating the function we ask you to write, the Gradescope autograder will **deduct** points from your public score, even if the way you did it produced the correct answer.

#### Required Functions:
- `download`
- `parse_html`

In this project, you will also be required to define certain **data structures**. If you do not create these data structures exactly as specified, the Gradescope autograder will **deduct** points from your public score, even if the way you did it produced the correct answer.

#### Required Data Structures:
- `rankings`
- `year_2021_ranking_df`
- `year_2022_ranking_df`
- `year_2023_ranking_df`
- `institutions_df`

In addition, you are also **required** to follow the requirements below:
* **Avoid using loops to iterate over pandas dataframes and instead use boolean indexing.**
* Do **not** use `loc` to look up data in **DataFrames** or **Series**, unless you are explicitly told to do so. You are **allowed** to use `iloc`.
* Do **not** use **absolute** paths such as `C://mdoescher//cs220//p12`. You may **only** use **relative paths**.
* Do **not** leave irrelevant output or test code that we didn't ask for.
* **Avoid** calling **slow** functions multiple times within a loop.
* Do **not** define multiple functions with the same name or define multiple versions of one function with different names. Just keep the best version.

### Section 1: Exploring `rankings.json`

#### Download `rankings.json` and create `rankings` Dataframe

Use `download` to pull the data from here (**do not manually download**): https://cs220.cs.wisc.edu/projects/data/rankings.json and store it in the file `rankings.json`. Since you already worked with this same file in the lab you should already have `rankings.json`, but just incase you should call the `download` function again. Once you are sure you have the file, create a Dataframe `rankings` from this file. 

Points possible: 4.0

In [74]:
# use the 'download' function to download the data from the webpage
url = "https://cs220.cs.wisc.edu/projects/data/rankings.json"
filename = "rankings.json"
download(url, filename)
# to the file 'rankings.json'
rankings = pd.read_json(filename)

# open 'rankings.json' with pd.read_json('rankings.json') and store in the variable 'rankings'

In [75]:
student_grader.check("download-rankings", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for download-rankings...
Great job! You passed all test cases for this question.


True

#### Question 1: How **many** countries do we have in our dataset?

Your output **must** be an **int** representing the number of *unique* countries in the dataset.

Points possible: 4.0

In [76]:
# compute and store the answer in the variable 'num_countries', then display it
num_countries = rankings["Country"].nunique()
print(num_countries)

98


In [77]:
student_grader.check("q1", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q1...
98
Great job! You passed all test cases for this question.


True

#### Question 2: Generate a `pandas` **DataFrame** containing **all** the statistics of the **highest-ranked** institution based on `World Rank` across all the years.

Your output **must** be a pandas **DataFrame** with 3 rows and 10 columns. It **must** contain all the data for the institutions with `World Rank` of *1*. It **must** look like this:

||**Year**|**World Rank**|**Institution**|**Country**|**National Rank**|**Education Rank**|**Employability Rank**|**Faculty Rank**|**Research Rank**|**Score**|
|---|---|---|---|---|---|---|---|---|---|---|
|**0**|2021|1|Harvard University|USA|1|1.0|1.0|1.0|1.0|100.0|
|**2000**|2022|1|Harvard University|USA|1|1.0|1.0|1.0|1.0|100.0|
|**4000**|2023|1|Harvard University|USA|1|1.0|1.0|1.0|1.0|100.0|

Points possible: 4.0

In [78]:
# compute and store the answer in the variable 'highest_ranked', then display it
highest_ranked = rankings[rankings["World Rank"] ==1]
print(highest_ranked)

      Year  World Rank         Institution Country  National Rank  \
0     2021           1  Harvard University     USA              1   
2000  2022           1  Harvard University     USA              1   
4000  2023           1  Harvard University     USA              1   

      Education Rank  Employability Rank  Faculty Rank  Research Rank  Score  
0                1.0                 1.0           1.0            1.0  100.0  
2000             1.0                 1.0           1.0            1.0  100.0  
4000             1.0                 1.0           1.0            1.0  100.0  


In [79]:
student_grader.check("q2", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q2...
      Year  World Rank         Institution Country  National Rank  \
0     2021           1  Harvard University     USA              1   
2000  2022           1  Harvard University     USA              1   
4000  2023           1  Harvard University     USA              1   

      Education Rank  Employability Rank  Faculty Rank  Research Rank  Score  
0                1.0                 1.0           1.0            1.0  100.0  
2000             1.0                 1.0           1.0            1.0  100.0  
4000             1.0                 1.0           1.0            1.0  100.0  
Great job! You passed all test cases for this question.


True

#### Question 3: Generate a `pandas` **DataFrame** containing **all** the statistics of *University of Wisconsin–Madison*.

**Hint**: The `–` symbol in the text above is not the regular hyphen (`-`) symbol. It is recommended that you just *copy/paste* the string `'University of Wisconsin–Madison'` into your code instead of typing it yourself.

Your output **must** be a pandas **DataFrame** with 3 rows and 10 columns. It **must** look like this:

||**Year**|**World Rank**|**Institution**|**Country**|**National Rank**|**Education Rank**|**Employability Rank**|**Faculty Rank**|**Research Rank**|**Score**|
|---|---|---|---|---|---|---|---|---|---|---|
|**24**|2021|25|University of Wisconsin–Madison|USA|19|33.0|97.0|29.0|32.0|87.3|
|**2026**|2022|27|University of Wisconsin–Madison|USA|20|34.0|100.0|30.0|35.0|87.0|
|**4027**|2023|28|University of Wisconsin–Madison|USA|20|36.0|102.0|30.0|41.0|87.0|

Points possible: 4.0

In [80]:
# compute and store the answer in the variable 'uw_madison', then display it
uw_madison = rankings[rankings["Institution"] == "University of Wisconsin–Madison"]
print(uw_madison)

      Year  World Rank                      Institution Country  \
24    2021          25  University of Wisconsin–Madison     USA   
2026  2022          27  University of Wisconsin–Madison     USA   
4027  2023          28  University of Wisconsin–Madison     USA   

      National Rank  Education Rank  Employability Rank  Faculty Rank  \
24               19            33.0                97.0          29.0   
2026             20            34.0               100.0          30.0   
4027             20            36.0               102.0          30.0   

      Research Rank  Score  
24             32.0   87.3  
2026           35.0   87.0  
4027           41.0   87.0  


In [81]:
student_grader.check("q3", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q3...
      Year  World Rank                      Institution Country  \
24    2021          25  University of Wisconsin–Madison     USA   
2026  2022          27  University of Wisconsin–Madison     USA   
4027  2023          28  University of Wisconsin–Madison     USA   

      National Rank  Education Rank  Employability Rank  Faculty Rank  \
24               19            33.0                97.0          29.0   
2026             20            34.0               100.0          30.0   
4027             20            36.0               102.0          30.0   

      Research Rank  Score  
24             32.0   87.3  
2026           35.0   87.0  
4027           41.0   87.0  
Great job! You passed all test cases for this question.


True

#### Question 4: What is the `National Rank` of the *University of Wisconsin–Madison* in the `Year` *2023*?

Your output **must** be an **int**. You **must** use **Boolean indexing** on the variable `uw_madison` (from the previous question) to answer this question.

**Hint:** Use Boolean indexing on the DataFrame `uw_madison` to find the data for the year *2023*. You may then extract the `National Rank` column from the subset DataFrame. Finally, use `iloc` to lookup the value in the DataFrame which contains only one row and one column.

Points possible: 4.0

In [82]:
uw_madison_2023 = uw_madison[uw_madison["Year"] == 2023]
uw_madison_nat_rank = uw_madison_2023["National Rank"].iloc[0]
print(uw_madison_nat_rank)

20


In [83]:
student_grader.check("q4", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q4...
20
Great job! You passed all test cases for this question.


True

#### Question 5: What is the **average** `Score` of the *University of Wisconsin–Madison*?

Your output **must** be a **float**. You **must** use the variable `uw_madison` to answer this question.

**Hint:** You **must** extract the `Score` column of the **DataFrame** `uw_madison` as a **Series**. You can find the **average** of  all the scores in a **Series** with the `Series.mean` function.

Points possible: 4.0

In [84]:
# compute and store the answer in the variable 'uw_madison_avg_score', then display it
uw_madison_avg_score = uw_madison["Score"].mean()
print(uw_madison_avg_score)

87.10000000000001


In [85]:
student_grader.check("q5", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q5...
87.10000000000001
Great job! You passed all test cases for this question.


True

#### Question 6: Generate a `pandas` **DataFrame** containing **all** the statistics of universities from the `Country` *Singapore* in the `Year` *2022*.

Your output **must** be a pandas **DataFrame** with 4 rows and 10 columns. It **must** look like this:

||**Year**|**World Rank**|**Institution**|**Country**|**National Rank**|**Education Rank**|**Employability Rank**|**Faculty Rank**|**Research Rank**|**Score**|
|---|---|---|---|---|---|---|---|---|---|---|
| **2084** | 2022 | 85 | National University of Singapore | Singapore | 1 | 342.0 | 172.0 | NaN | 41.0 | 82.4 |
| **2132** | 2022 | 133 | Nanyang Technological University | Singapore | 2 | NaN | 843.0 | NaN | 66.0 | 80.5 |
| **3011** | 2022 | 1012 | Singapore University of Technology and Design | Singapore | 3 | NaN | NaN | NaN | 965.0 | 70.2 |
| **3336** |   2022 |         1337 | Singapore Management University | Singapore | 4 | NaN | NaN | NaN | 1269.0 | 68.4 |

**Hint:** When there are **multiple** conditions to filter a **DataFrame**, you can combine all the conditions with `&` as a logical operator between them. For example, you can extract the data for all the institutions with `Education Rank <= 10` and `Faculty Rank <= 10` with:

```python
rankings[(rankings["Education Rank"] <= 10) & (rankings["Faculty Rank"] <= 10)]
```

Points possible: 4.0

In [86]:
# compute and store the answer in the variable 'singapore_inst', then display it
singapore_inst = rankings[(rankings["Country"] == "Singapore") & (rankings["Year"] == 2022)]
print(singapore_inst)

      Year  World Rank                                    Institution  \
2084  2022          85               National University of Singapore   
2132  2022         133               Nanyang Technological University   
3011  2022        1012  Singapore University of Technology and Design   
3336  2022        1337                Singapore Management University   

        Country  National Rank  Education Rank  Employability Rank  \
2084  Singapore              1           342.0               172.0   
2132  Singapore              2             NaN               843.0   
3011  Singapore              3             NaN                 NaN   
3336  Singapore              4             NaN                 NaN   

      Faculty Rank  Research Rank  Score  
2084           NaN           41.0   82.4  
2132           NaN           66.0   80.5  
3011           NaN          965.0   70.2  
3336           NaN         1269.0   68.4  


In [87]:
student_grader.check("q6", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q6...
      Year  World Rank                                    Institution  \
2084  2022          85               National University of Singapore   
2132  2022         133               Nanyang Technological University   
3011  2022        1012  Singapore University of Technology and Design   
3336  2022        1337                Singapore Management University   

        Country  National Rank  Education Rank  Employability Rank  \
2084  Singapore              1           342.0               172.0   
2132  Singapore              2             NaN               843.0   
3011  Singapore              3             NaN                 NaN   
3336  Singapore              4             NaN                 NaN   

      Faculty Rank  Research Rank  Score  
2084           NaN           41.0   82.4  
2132           NaN           66.0   80.5  
3011           NaN          965.0   70.2  
3336           NaN         1

True

#### Question 7:  In the `Year` *2022*, what was the **highest-ranked** institution in the `Country` *Germany*?

Your output **must** be a **string** representing the **name** of this institution.

**Hint:** The highest-ranked institution in *Germany* is the institution from Germany with a `National Rank` of *1*.

Points possible: 4.0

In [88]:
# compute and store the answer in the variable 'german_best_name', then display it
germany_2022 = rankings[(rankings["Country"] == "Germany") & (rankings["Year"] == 2022)]
german_best_name = germany_2022[germany_2022["National Rank"] == 1]["Institution"].iloc[0]
print(german_best_name)

Ludwig Maximilian University of Munich


In [89]:
student_grader.check("q7", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q7...
Ludwig Maximilian University of Munich
Great job! You passed all test cases for this question.


True

#### Question 8: In the `Year` *2022*, list **all** the institutions in the *USA* that were ranked **better** than the highest-ranked institution in *Germany*.

Your output **must** be a **list** containing the **names** of all universities from *USA* with a **better** `World Rank` than the institution `german_best_name` in the `Year` *2022*. By **better** ranked, we refer to institutions with a **lower** value under the `World Rank` column.

**Hint:** You could store the entire row of the highest ranked institution from Germany in a different variable in Question 7, and use it to extract its `World Rank`. You could go back to your answer for Question 7, and edit it slightly to do this.

Points possible: 4.0

In [91]:
# compute and store the answer in the variable 'us_better_than_german_best', then display it
germany_2022 = rankings[(rankings["Country"] == "Germany") & (rankings["Year"] == 2022)]
german_best = germany_2022[germany_2022["National Rank"] ==1]
german_best_world_rank = german_best["World Rank"].iloc[0]
us_better_than_german_best = rankings[
    (rankings["Country"] == "USA") &
    (rankings["Year"] == 2022) &
    (rankings["World Rank"] < german_best_world_rank)
]["Institution"].tolist()

print(us_better_than_german_best)

['Harvard University', 'Massachusetts Institute of Technology', 'Stanford University', 'Princeton University', 'University of Chicago', 'Columbia University', 'University of Pennsylvania', 'California Institute of Technology', 'Yale University', 'University of California, Berkeley', 'Cornell University', 'University of Michigan, Ann Arbor', 'Johns Hopkins University', 'Northwestern University', 'University of California, Los Angeles', 'Duke University', 'University of Illinois at Urbana–Champaign', 'New York University', 'University of Washington', 'University of Wisconsin–Madison', 'University of Texas at Austin', 'University of California, San Diego', 'University of California, San Francisco', 'University of North Carolina at Chapel Hill', 'Dartmouth College']


In [92]:
student_grader.check("q8", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q8...
['Harvard University', 'Massachusetts Institute of Technology', 'Stanford University', 'Princeton University', 'University of Chicago', 'Columbia University', 'University of Pennsylvania', 'California Institute of Technology', 'Yale University', 'University of California, Berkeley', 'Cornell University', 'University of Michigan, Ann Arbor', 'Johns Hopkins University', 'Northwestern University', 'University of California, Los Angeles', 'Duke University', 'University of Illinois at Urbana–Champaign', 'New York University', 'University of Washington', 'University of Wisconsin–Madison', 'University of Texas at Austin', 'University of California, San Diego', 'University of California, San Francisco', 'University of North Carolina at Chapel Hill', 'Dartmouth College']
Great job! You passed all test cases for this question.


True

#### Question 9: What is the **highest-ranked** institution based on `Education Rank` in *China* for the `Year` *2023*?

Your output **must** be a **string** representing the **name** of this institution. You may **assume** there is only one institution satisfying these requirements. By the **highest-ranked** institution, we refer to the institution with the **least** value under the `Education Rank` column.

**Hint:** You can find the **minimum** value in a **Series** with the `Series.min` method. You can find the documentation [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.min.html) or by executing the line `help(pd.Series.min)` in a separate cell below.

Points possible: 4.0

In [93]:
# compute and store the answer in the variable 'china_highest_qoe', then display it
china_2023 = rankings[(rankings["Country"] == "China") & (rankings["Year"] == 2023)]
min_education_rank = china_2023["Education Rank"].min()
china_highest_qoe = china_2023[china_2023["Education Rank"] == min_education_rank]["Institution"].iloc[0]
print(china_highest_qoe)

Tsinghua University


In [94]:
student_grader.check("q9", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q9...
Tsinghua University
Great job! You passed all test cases for this question.


True

#### Question 10: What are the **top** *five* **highest-ranked** institutions based on `Research Rank` in *India* for the `Year` *2022*?

Your output **must** be a **list** of institutions **sorted** in *increasing* order of their `Research Rank`.

**Hint:** For sorting a DataFrame based on the values of a particular column, you can use the `DataFrame.sort_values(by="column_name")` method (where `column_name` is the column on which you want to sort). You can find the documentation [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html) or by executing the line `help(pd.Series.sort_values)` in a separate cell below.

Points possible: 4.0

In [95]:
# compute and store the answer in the variable 'india_highest_research', then display it
india_2022 = rankings[(rankings["Country"] == "India") & (rankings["Year"] == 2022)]
sorted_india_research = india_2022.sort_values(by="Research Rank")
india_highest_research = sorted_india_research.head(5)["Institution"].tolist()
print(india_highest_research)

['Indian Institute of Science', 'Tata Institute of Fundamental Research', 'Indian Institute of Technology Bombay', 'Indian Institute of Technology Madras', 'University of Delhi']


In [96]:
student_grader.check("q10", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q10...
['Indian Institute of Science', 'Tata Institute of Fundamental Research', 'Indian Institute of Technology Bombay', 'Indian Institute of Technology Madras', 'University of Delhi']
Great job! You passed all test cases for this question.


True

### Section 2: Data Across the Years

For the next few questions, we will be analyzing how the rankings of the institutions change across the three years in the dataset. As you might have already noticed, the list of institutions in each year's rankings are different. As a result, for several institutions in the dataset, we do not have the rankings for all three years. Since it will be more challenging to analyze such institutions, we will simply skip them.

#### Question 11: How **many** institutions have rankings for **all** three years?

Your output **must** be an **integer**. To get started, you have been provided with a code snippet below.

**Hint:** You could make **sets** of the institutions that appear in each **DataFrame**, and find their **intersection**. Look up how to find the intersection of two or more sets in Python, on the internet!

Points possible: 4.0

In [97]:
# compute and store the answer in the variable 'num_institutions_2021_2022_2023', then display it
# replace the ... with your code

year_2021_ranking_df = rankings[rankings["Year"] == 2021]
year_2022_ranking_df = rankings[rankings["Year"] == 2022]
year_2023_ranking_df = rankings[rankings["Year"] == 2023]

# TODO: make sets of the institutions in each of the three years
institutions_2021 = set(year_2021_ranking_df["Institution"])
institutions_2022 = set(year_2022_ranking_df["Institution"])
institutions_2023 = set(year_2023_ranking_df["Institution"])
# TODO: find the intersection of the three sets
institutions_2021_2022_2023 = institutions_2021 & institutions_2022 & institutions_2023
# TODO: find the length of the intersection
num_institutions_2021_2022_2023 = len(institutions_2021_2022_2023)

num_institutions_2021_2022_2023

1897

In [98]:
student_grader.check("q11", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q11...
Great job! You passed all test cases for this question.


True

#### Data Structure 1: `institutions_df`

You are now going to create a new **DataFrame** with a **unique** list of institutions which have featured in the rankings for **all** three years, along with their `World Rank` across the three years. Specifically, the **DataFrame** **must** have the following four columns - `'Institution'`, `'2021 ranking'`, `'2022 ranking'`, and `'2023 ranking'`.

Points possible: 4.0

In [99]:
# define the variable 'institutions_df', but do NOT display it here
institutions_list = []
for institution in institutions_2021_2022_2023:
    institution_data = {
        "Institution": institution,
        "2021 ranking": year_2021_ranking_df[year_2021_ranking_df["Institution"] == institution]["World Rank"].iloc[0],
        "2022 ranking": year_2022_ranking_df[year_2022_ranking_df["Institution"] == institution]["World Rank"].iloc[0],
        "2023 ranking": year_2023_ranking_df[year_2023_ranking_df["Institution"] == institution]["World Rank"].iloc[0],
    }
    institutions_list.append(institution_data)
institutions_df = pd.DataFrame(institutions_list)
# TODO: initalize an empty list to store the list of institutions
# TODO: loop through the variable 'institutions_2021_2022_2023' defined above
    # TODO: create a new dictionary with the necessary key/value pairs
    # TODO: append the dictionary to the list
# TODO: create the DataFrame from the list of dictionaries

In [100]:
student_grader.check("institutions_df", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for institutions_df...
Great job! You passed all test cases for this question.


True

#### Question 12: Between the years *2022* and *2023*, **list** the institutions which have seen an **improvement** in their `World Rank` by **more than** *200* ranks.

Your output **must** be a **list** of institution names. The **order** does **not** matter. You **must** use the DataFrame `institutions_df` to answer this question.

**Hints:**

1. In pandas, subtraction of two columns can be simply done using subtraction(`-`) operator. For example,
``` python
df["difference"] = df["column1"] - df["column2"]
```
will create a *new column* `difference` with the difference of the values from the columns `column1` and `column2`.
2. Note that an *improved* ranking means that the `World Rank` has *decreased*.

# compute and store the answer in the variable 'improved_institutions', then display it
institutions_df["difference"] = institutions_df["2022 ranking"] - institutions_df["2023 ranking"]
improved_institutions = institutions_df[institutions_df["difference"] > 200]["Institution"].tolist()
improved_institutionsPoints possible: 4.0

In [101]:
# compute and store the answer in the variable 'improved_institutions', then display it
institutions_df["difference"] = institutions_df["2022 ranking"] - institutions_df["2023 ranking"]
improved_institutions = institutions_df[institutions_df["difference"] > 200]["Institution"].tolist()
improved_institutions

['University of Reims Champagne-Ardenne',
 'University of Picardie Jules Verne',
 'Okinawa Institute of Science and Technology',
 'Addis Ababa University',
 'University of Western Brittany',
 'University of Caen Normandy',
 'University of Technology of Belfort-Montbéliard']

In [102]:
student_grader.check("q12", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q12...
Great job! You passed all test cases for this question.


True

#### Question 13: Between the years *2021* and *2023*, which institution had the **third largest** change in its `World Rank`?

Your output **must** be a **string** representing the name of the institution with the **third greatest absolute difference** between its `World Rank` in 2021 and 2023. You **must** use the DataFrame `institutions_df` to answer this question. Feel free add a new column to the DataFrame, so long as you do not modify the existing data.

Points possible: 4.0

In [103]:
# compute and store the answer in the variable 'third_most_change_inst', then display it
institutions_df["absolute_change"] = abs(institutions_df["2021 ranking"] - institutions_df["2023 ranking"])
sorted_institutions = institutions_df.sort_values(by="absolute_change", ascending = False)
third_most_change_inst = sorted_institutions.iloc[2]["Institution"]
third_most_change_inst

'Huzhou University'

In [104]:
student_grader.check("q13", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q13...
Great job! You passed all test cases for this question.


True

#### Question 14: For all the three years, find the **number** of institutions that **improved** their `World Rank` between **each year** by **at least** 5 ranks.

Your output **must** be an **integer** representing the number of institutions whose `World Rank` **decreased** each year by **at least** 5 ranks. You **must** use the DataFrame `institutions_df` to answer this question.

Points possible: 5.0

In [105]:
# compute and store the answer in the variable 'five_improved', then display it
institutions_df["improvement_2021_2022"] = institutions_df["2021 ranking"] - institutions_df["2022 ranking"]
institutions_df["improvement_2022_2023"] = institutions_df["2022 ranking"] - institutions_df["2023 ranking"]
improved_institutions = institutions_df[
    (institutions_df["improvement_2021_2022"] >= 5) & 
    (institutions_df["improvement_2022_2023"] >= 5)
]

five_improved = len(improved_institutions)
five_improved

463

In [106]:
student_grader.check("q14", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q14...
Great job! You passed all test cases for this question.


True

#### Question 15: In the `Year` *2021*, **list** the institutions which do **not** feature in the **top** *50* in the world based on `World Rank`, but have a `Employability Rank` **less than or equal** to *25*.

Your output **must** be a **list** of institutions. The **order** does **not** matter. You **must** use the `year_2021_ranking_df` DataFrame that you created in Question 11 to answer this question.

Points possible: 5.0

In [107]:
# compute and store the answer in the variable 'only_top_employability', then display it
only_top_employability = year_2021_ranking_df[
    (year_2021_ranking_df["World Rank"] > 50) & 
    (year_2021_ranking_df["Employability Rank"] <= 25)
]["Institution"].tolist()
print(only_top_employability)

['Keio University', 'INSEAD', "École nationale d'administration", 'HEC Paris', 'China Europe International Business School', 'International Institute for Management Development', 'École des ponts ParisTech', 'Indian Institute of Management Ahmedabad', 'Stockholm School of Economics', 'Hitotsubashi University', 'Central Party School of the Communist Party of China', 'Graduate Faculty, General Research Institute For Nonferrous Metals']


In [108]:
student_grader.check("q15", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q15...
['Keio University', 'INSEAD', "École nationale d'administration", 'HEC Paris', 'China Europe International Business School', 'International Institute for Management Development', 'École des ponts ParisTech', 'Indian Institute of Management Ahmedabad', 'Stockholm School of Economics', 'Hitotsubashi University', 'Central Party School of the Communist Party of China', 'Graduate Faculty, General Research Institute For Nonferrous Metals']
Great job! You passed all test cases for this question.


True

#### Question 16: **List** the universities which ranked in the **top** 50 of world rankings (`World Rank`) in the `Year` *2021* but **failed** to do so in the `Year` *2023*.

Your output **must** be a **list** of institutions. The **order** does **not** matter. You **must** use the `year_2021_ranking_df` and `year_2023_ranking_df` DataFrames that you created in Question 11 to answer this question.

**Hints:**
1. There could be institutions that are ranked in the **top** 50 in *2021* but do not feature in *2023* at all; you still want to include them in your list.
2. You can use `sort_values` and `iloc` to identify the **top** 50 institutions.
3. Given two *sets* `A` and `B`, you can find the elements which are in `A` but not in `B` using `A - B`. For example,
```python
set_A = {10, 20, 30, 40, 50}
set_B = {20, 40, 70}
set_A - set_B == {10, 30, 50} # elements which are in set_A but not in set_B
```

Points possible: 5.0

In [109]:
# compute and store the answer in the variable 'top_50_only_2021', then display it
top_50_2021 = set(year_2021_ranking_df[year_2021_ranking_df["World Rank"] <= 50]["Institution"])
top_50_2023 = set(year_2023_ranking_df[year_2023_ranking_df["World Rank"] <= 50]["Institution"])
top_50_only_2021 = list(top_50_2021 - top_50_2023)
print(top_50_only_2021)

['Rockefeller University', 'University of Washington - Seattle', 'Rutgers University–New Brunswick', 'University of British Columbia', 'University of Paris']


In [110]:
student_grader.check("q16", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q16...
['Rockefeller University', 'University of Washington - Seattle', 'Rutgers University–New Brunswick', 'University of British Columbia', 'University of Paris']
Great job! You passed all test cases for this question.


True

#### Question 17: **List** the countries which have **at least** *5* and **at most** *10* institutions featuring in the **top** *100* of world rankings (`World Rank`) in the `Year` *2023*.

Your output **must** be a **list**.

**Hints:**

1. In a **DataFrame**, to find the **number** of times each unique value in a column repeats, you can use the `DataFrame.value_counts` method. For example,
``` python
rankings["Country"].value_counts()
```
would output a `pandas` **Series** with the **indices** being the unique values of `Country` and the **values** being the **number** of times each country has featured in the `rankings` **DataFrame**. You can find the documentation [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.value_counts.html) or by using the `help` function in a separate cell. You can adapt this code to find the number of institutions from each country that features in the `Year` *2023*.
2. Just like with **DataFrames**, you can use Boolean indexing on **Series**. For example, try something like this in a separate cell below:
```python
a = pd.Series([100, 200, 300])
a[a > 100]
```
3. You can extract the **indices** of a **Series**, `s` with `s.index`.

Points possible: 5.0

In [111]:
# compute and store the answer in the variable 'almost_top_countries', then display it
top_100_2023 = year_2023_ranking_df[year_2023_ranking_df["World Rank"] <= 100]
country_counts = top_100_2023["Country"].value_counts()
almost_top_countries = country_counts[(country_counts >= 5) & (country_counts <= 10)].index.tolist()
print(almost_top_countries)

['United Kingdom', 'China', 'Germany', 'France']


In [112]:
student_grader.check("q17", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q17...
['United Kingdom', 'China', 'Germany', 'France']
Great job! You passed all test cases for this question.


True

### Section 3: BeautifulSoup

In real life, you don't often have data in nice JSON format like `rankings.json`. Instead, data needs to be *scraped* from multiple webpages and requires some cleanup before it can be used.

Most of the projects in CS220 have used data obtained via web scraping, including this one. For p12, as explained above, we obtained the data by scraping the following websites:

* https://cwur.org/2020-21.php
* https://cwur.org/2021-22.php
* https://cwur.org/2023.php

Our `rankings.json` file was created using data from these webpages. For the rest of this project, you will write the code to **recreate** `rankings.json` file from the tables in these html pages yourself! We also do **not** want all students in this class to be making multiple requests to the webpages above, as that could be very costly for the people managing the webpages. Instead, we have made **copies** of the webpages above, which can be found here:

* https://cs220.cs.wisc.edu/projects/data/2021.html
* https://cs220.cs.wisc.edu/projects/data/2022.html
* https://cs220.cs.wisc.edu/projects/data/2023.html

Before you can parse these html files, you must first *download* them. You **must** use your `download` function to download these files.

#### Download `2021.html`, `2022.html` and `2023.html`

In the lab portion, you downloaded the files `2021.html`, `2022.html` and `2023.html`. Just in case they are no longer in your directory, use the `download` function you previously created to download the contents of each of the URLs above and save them into files. Name the files `2021.html`, `2022.html` and `2023.html` based on the respective URL if you need to redownload them.

Points possible: 5.0

In [113]:
# use the 'download' function to download the data from the webpage and save the return value in 'message_2021'
# 'https://cs220.cs.wisc.edu/projects/data/2021.html'
# to the file '2021.html'
message_2021 = download("https://cs220.cs.wisc.edu/projects/data/2021.html", "2021.html")

# use the 'download' function to download the data from the webpage and save the return value in 'message_2022'
# 'https://cs220.cs.wisc.edu/projects/data/2022.html'
# to the file '2022.html'
message_2022 = download("https://cs220.cs.wisc.edu/projects/data/2022.html", "2022.html")

# use the 'download' function to download the data from the webpage and save the return value in 'message_2023'
# 'https://cs220.cs.wisc.edu/projects/data/2023.html'
# to the file '2023.html'
message_2023 = download("https://cs220.cs.wisc.edu/projects/data/2023.html", "2023.html")

print(message_2021)
print(message_2022)
print(message_2023)

2021.html already exists!
2022.html already exists!
2023.html already exists!


In [114]:
student_grader.check("downloads", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for downloads...
2021.html already exists!
2022.html already exists!
2023.html already exists!
Great job! You passed all test cases for this question.


True

#### Question 18: Use `BeautifulSoup` to **parse** `2021.html`, and find the **table** containing the ranking data. 

Extract the **column names** of this table and the first row of the table to create a **dictionary** where the column headers are the keys and the corresponding values are extracted from the **first** row.

You do **not** have to perform any typecasting of the data **yet**. Your output **must** be a **dictionary** having the format as given below:
```python
{
    'World Rank': '1',
    'Institution': 'Harvard University',
    'Country': 'USA',
    'National Rank': '1',
    'Education Rank': '1',
    'Employability Rank': '1',
    'Faculty Rank': '1',
    'Research Rank': '1',
    'Score': '100'
}
```

**Hints:** You can use the `find` or `find_all` **methods** to identify the table and its header. Or you can use the `parse_html` function you defined above.

Points possible: 5.0

In [115]:
# compute and store the answer in the variable 'first_dict', then display it
with open("2021.html", "r", encoding="utf-8") as file:
    file_content = file.read()
soup = BeautifulSoup(file_content, "html.parser")
table = soup.find("table")
header_tags = table.find_all("th")
headers = [tag.get_text().strip() for tag in header_tags]
first_row = table.find_all("tr")[1]  
cells = first_row.find_all("td")
values = [cell.get_text().strip() for cell in cells]
first_dict = dict(zip(headers, values))

print(first_dict)

{'World Rank': '1', 'Institution': 'Harvard University', 'Country': 'USA', 'National Rank': '1', 'Education Rank': '1', 'Employability Rank': '1', 'Faculty Rank': '1', 'Research Rank': '1', 'Score': '100'}


In [116]:
student_grader.check("q18", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q18...
{'World Rank': '1', 'Institution': 'Harvard University', 'Country': 'USA', 'National Rank': '1', 'Education Rank': '1', 'Employability Rank': '1', 'Faculty Rank': '1', 'Research Rank': '1', 'Score': '100'}
Great job! You passed all test cases for this question.


True

#### Question 19: Calculate the **average** score of the **first** 5 institutions in the file `2021.html`.

Your output **must** be a **float** calculated by averaging the scores from the first 5 dictionaries in the file. You **must** use the `parse_html` function to parse the file, and **slice** the list such that you would only loop through the **first five** institutions. For each **dictionary** in the **list** you must use the `Score` key to get the score for that particular institution.

Points possible: 5.0

In [117]:
# compute and store the answer in the variable 'avg_top_5', then display it
data_2021 = parse_html("2021.html")
top_5_institutions = data_2021[:5]
avg_top_5 = sum(float(inst["Score"]) for inst in top_5_institutions) / len(top_5_institutions)

print(avg_top_5)

95.84


In [118]:
student_grader.check("q19", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q19...
95.84
Great job! You passed all test cases for this question.


True

#### Question 20: Parse the contents of the **three** files `2021.html`, `2022.html`, and `2023.html` and combine them to create a **single** file named `my_rankings.json`.

You **must** create a **file** named `my_rankings.json` in your current directory. The contents of this file **must** be **identical** to `rankings.json`.

**Hints:**
1. Using the logic from the question above, combine the data from these three files into a single list of dicts, and write it into the file `"my_rankings.json"`.
2. You can use the `write_json` function that was introduced in lecture.

Points possible: 5.0

In [119]:
# the 'write_json' function from lecture has been provided for you here
def write_json(path, data):
    with open(path, 'w', encoding = "utf-8") as f:
        json.dump(data, f, indent = 2)
data_2021 = parse_html("2021.html")
data_2022 = parse_html("2022.html")
data_2023 = parse_html("2023.html")
my_rankings = data_2021 + data_2022 + data_2023
write_json("my_rankings.json", my_rankings)
print("my_rankings.json has been created!")
# define a variable that contains the combined contents of the individual html files called 'my_rankings'
# `my_rankings` should be a list of dictionaries

# parse the three files with parse_html and write the contents into 'my_rankings.json'


my_rankings.json has been created!


In [120]:
student_grader.check("q20", should_get_llm_feedback=False)

Make sure you saved the notebook before running this cell. Running check for q20...
my_rankings.json has been created!
Great job! You passed all test cases for this question.


True

### Submission

Make sure you have run all cells in your notebook in order before submitting on Gradescope. Your notebook should not contain any uncaught Exceptions, otherwise the Gradescope autograder will not give you any points.